B) Implementação Computacional:

Usando linguagens de programação (Python, ou Matalb ou Octave, etc) implementar exemplos de modelos considerando:

3. Elementos bidimensionais (estado plano de tensões)

Descrição da Implementação — Elemento 2D (Estado Plano de Tensões)
1. Definição da geometria e conectividade
* Foram definidos 3 nós formando um elemento triangular:
  * Nó 1: (0.0, 0.0)
  * Nó 2: (1.0, 0.0)
  * Nó 3: (0.0, 1.0)

* A conectividade foi definida como um único elemento triangular com vértices nos nós 1, 2 e 3.

2. Propriedades do material e espessura
* Módulo de elasticidade E = 210 x 10^9 Pa
* Coeficiente de Poisson v = 0,3
* Espessura do elemento t = 0,01 m

3. Matriz constitutiva D
* Foi calculada para estado plano de tensões, considerando o comportamento elástico linear do material.

4. Cálculo da matriz B e área do elemento
* A função B_matrix calcula:
  * A matriz de deformações B com base nas coordenadas dos nós
  * A área do triângulo usando determinantes geométricos

5. Montagem da matriz de rigidez global
* A matriz de rigidez local K_e foi calculada como:

K_e = t ⋅ A ⋅ B^T ⋅ D ⋅ B
* Os graus de liberdade de cada nó foram mapeados e a matriz local foi somada à matriz global K

6. Aplicação de carregamento
* Foi aplicada uma força vertical de –1000 N no eixo y
 do nó 2

7. Condições de contorno
* Os nós 1 e 3 foram fixados (graus de liberdade 0, 1, 4, 5)
* Apenas os graus de liberdade do nó 2 foram considerados livres

8. Resolução do sistema
* O sistema foi reduzido para os graus de liberdade livres
* Resolvido para obter os deslocamentos U_x e U_y de cada nó

9. Resultados
* O script imprime os deslocamentos em x e y para os três nós
* Apenas o nó 2 apresenta deslocamento, já que os outros estão fixos

In [ ]:
import numpy as np

# Coordenadas dos nós
nodes = np.array([
    [0.0, 0.0],  # Nó 1
    [1.0, 0.0],  # Nó 2
    [0.0, 1.0]   # Nó 3
])

# Conectividade
elements = [(0, 1, 2)]

# Propriedades
E = 210e9      # Pa
nu = 0.3
t = 0.01       # m

# Matriz constitutiva D para estado plano de tensões
D = (E / (1 - nu**2)) * np.array([
    [1, nu, 0],
    [nu, 1, 0],
    [0, 0, (1 - nu) / 2]
])

# Função para matriz B e área
def B_matrix(x1, y1, x2, y2, x3, y3):
    A = 0.5 * ((x2 - x1)*(y3 - y1) - (x3 - x1)*(y2 - y1))
    b1 = y2 - y3
    b2 = y3 - y1
    b3 = y1 - y2
    c1 = x3 - x2
    c2 = x1 - x3
    c3 = x2 - x1
    B = (1 / (2*A)) * np.array([
        [b1, 0, b2, 0, b3, 0],
        [0, c1, 0, c2, 0, c3],
        [c1, b1, c2, b2, c3, b3]
    ])
    return B, A

# Montagem da matriz de rigidez
K_global = np.zeros((6, 6))
for e in elements:
    i, j, k = e
    x1, y1 = nodes[i]
    x2, y2 = nodes[j]
    x3, y3 = nodes[k]
    B, A = B_matrix(x1, y1, x2, y2, x3, y3)
    Ke = t * A * B.T @ D @ B
    dofs = [2*i, 2*i+1, 2*j, 2*j+1, 2*k, 2*k+1]
    for m in range(6):
        for n in range(6):
            K_global[dofs[m], dofs[n]] += Ke[m, n]

# Vetor de forças (exemplo: força vertical no nó 2)
F = np.zeros(6)
F[3] = -1000  # N

# Condições de contorno (fixar nó 1 e nó 3)
fixed_dofs = [0, 1, 4, 5]
free_dofs = list(set(range(6)) - set(fixed_dofs))

# Resolver sistema
K_reduced = K_global[np.ix_(free_dofs, free_dofs)]
F_reduced = F[free_dofs]
U_reduced = np.linalg.solve(K_reduced, F_reduced)

# Reconstruir vetor de deslocamentos completo
U = np.zeros(6)
U[free_dofs] = U_reduced

print("Deslocamentos nos nós:")
for i in range(3):
    print(f"Nó {i+1}: Ux = {U[2*i]:.6e} m, Uy = {U[2*i+1]:.6e} m")

Deslocamentos nos nós:
Nó 1: Ux = 0.000000e+00 m, Uy = 0.000000e+00 m
Nó 2: Ux = 0.000000e+00 m, Uy = -2.476190e-06 m
Nó 3: Ux = 0.000000e+00 m, Uy = 0.000000e+00 m
